# PostgreSQL Database Connection Test

This notebook tests the connection to a PostgreSQL database using psycopg2.

# Import Python LIbraries

In [1]:
# Import required libraries
import psycopg2
from psycopg2 import OperationalError
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file if it exists
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


# Settings

In [2]:
# Database connection parameters
# You can modify these or set them in a .env file
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'kmc.tequila-ai.com'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'tequila_ai_reporting'),
    'user': os.getenv('DB_USER', 'james'),
    'password': os.getenv('DB_PASSWORD', ']dT1H-{ekquGfn^6'),
    'sslmode': os.getenv('DB_SSLMODE', 'require')  # SSL mode required for secure connection
}

print("Database Configuration:")
print(f"Host: {DB_CONFIG['host']}")
print(f"Port: {DB_CONFIG['port']}")
print(f"Database: {DB_CONFIG['database']}")
print(f"User: {DB_CONFIG['user']}")
print(f"Password: {'*' * len(DB_CONFIG['password']) if DB_CONFIG['password'] else 'Not set'}")
print(f"SSL Mode: {DB_CONFIG['sslmode']}")

Database Configuration:
Host: kmc.tequila-ai.com
Port: 5432
Database: tequila_ai_reporting
User: james
Password: ****************
SSL Mode: require


# Functions

In [3]:
# Test database connection
def test_connection():
    """Test the PostgreSQL database connection"""
    try:
        # Attempt to connect to the database
        print("Attempting to connect to PostgreSQL database...")
        connection = psycopg2.connect(**DB_CONFIG)
        
        # Create a cursor
        cursor = connection.cursor()
        
        # Execute a simple query to verify connection
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()
        
        print("\n✓ Connection successful!")
        print(f"\nPostgreSQL Database Version:")
        print(db_version[0])
        
        # Get current database name
        cursor.execute("SELECT current_database();")
        current_db = cursor.fetchone()[0]
        print(f"\nConnected to database: {current_db}")
        
        # Close cursor and connection
        cursor.close()
        connection.close()
        print("\n✓ Connection closed successfully")
        
        return True
        
    except OperationalError as e:
        print(f"\n✗ Connection failed!")
        print(f"Error: {e}")
        return False
    except Exception as e:
        print(f"\n✗ An unexpected error occurred!")
        print(f"Error: {e}")
        return False
    
# Create table from client_list dataframe and insert data
def create_and_insert_clientlist(df):
    """Create clientlist table based on dataframe structure and insert data"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        cursor = connection.cursor()
        
        # First, show dataframe info
        print("DataFrame Info:")
        print(f"Shape: {df.shape}")
        print(f"\nColumns: {list(df.columns)}")
        print(f"\nData types:\n{df.dtypes}")
        print(f"\nFirst few rows:")
        print(df.head())
        
        # Drop table if exists
        print("\n" + "="*50)
        print("Dropping existing 'clientlist' table if it exists...")
        cursor.execute("DROP TABLE IF EXISTS clientlist CASCADE;")
        
        # Create table with columns matching the dataframe
        print("Creating 'clientlist' table...")
        
        # Build CREATE TABLE statement dynamically based on dataframe columns
        # You may need to adjust data types based on your actual data
        create_table_sql = """
            CREATE TABLE clientlist (
                id SERIAL PRIMARY KEY,
        """
        
        # Add columns from dataframe (with proper quoting for special characters)
        for col in df.columns:
            dtype = df[col].dtype
            if dtype == 'object':
                sql_type = 'TEXT'
            elif dtype == 'int64':
                sql_type = 'INTEGER'
            elif dtype == 'float64':
                sql_type = 'NUMERIC'
            elif dtype == 'bool':
                sql_type = 'BOOLEAN'
            elif dtype == 'datetime64[ns]':
                sql_type = 'TIMESTAMP'
            else:
                sql_type = 'TEXT'
            
            # Quote column names to handle special characters
            create_table_sql += f'\n                "{col}" {sql_type},'
        
        # Remove last comma and close the statement
        create_table_sql = create_table_sql.rstrip(',') + "\n            );"
        
        print(f"\nSQL:\n{create_table_sql}")
        cursor.execute(create_table_sql)
        
        # Insert data
        print(f"\nInserting {len(df)} rows into 'clientlist' table...")
        
        # Prepare data for insertion
        columns = [f'"{col}"' for col in df.columns]  # Quote column names
        values = [tuple(row) for row in df.values]
        
        # Build INSERT statement
        insert_sql = f"""
            INSERT INTO clientlist ({', '.join(columns)})
            VALUES %s
        """
        
        execute_values(cursor, insert_sql, values)
        
        # Commit changes
        connection.commit()
        print(f"✓ Successfully inserted {len(df)} rows!")
        
        # Verify the data
        cursor.execute("SELECT COUNT(*) FROM clientlist;")
        count = cursor.fetchone()[0]
        print(f"\nVerification: Table now contains {count} rows")
        
        cursor.close()
        connection.close()
        
        return True
        
    except Exception as e:
        print(f"✗ Error: {e}")
        if connection:
            connection.rollback()
        return False

# Delete tables
def delete_tables():
    """Delete the sample tables"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        cursor = connection.cursor()
        
        # Drop tables in reverse order (due to foreign key constraints)
        print("Dropping 'orders' table...")
        cursor.execute("DROP TABLE IF EXISTS orders CASCADE;")
        
        print("Dropping 'products' table...")
        cursor.execute("DROP TABLE IF EXISTS products CASCADE;")
        
        print("Dropping 'customers' table...")
        cursor.execute("DROP TABLE IF EXISTS customers CASCADE;")
        
        # Commit the changes
        connection.commit()
        print("\n✓ All tables deleted successfully!")
        
        cursor.close()
        connection.close()
        
    except Exception as e:
        print(f"✗ Error deleting tables: {e}")
        if connection:
            connection.rollback()

# List all tables in the database
def list_tables():
    """List all tables in the current database"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        cursor = connection.cursor()
        
        # Query to get all tables in the public schema
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """)
        
        tables = cursor.fetchall()
        
        if tables:
            print(f"Found {len(tables)} table(s) in the database:\n")
            for i, table in enumerate(tables, 1):
                print(f"{i}. {table[0]}")
        else:
            print("No tables found in the database.")
        
        cursor.close()
        connection.close()
        
        return [table[0] for table in tables]
        
    except Exception as e:
        print(f"Error listing tables: {e}")
        return []

# Import data from clientlist table into a dataframe
def import_clientlist_from_db():
    """Read clientlist table from database into a pandas dataframe"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        
        # Use pandas read_sql to import data
        query = "SELECT * FROM clientlist;"
        
        print("Reading data from 'clientlist' table...")
        df = pd.read_sql(query, connection)
        
        print(f"✓ Successfully loaded {len(df)} rows from database")
        print(f"\nDataFrame Info:")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        print(f"\nFirst few rows:")
        print(df.head(10))
        
        print(f"\nData types:")
        print(df.dtypes)
        
        connection.close()
        
        return df
        
    except Exception as e:
        print(f"✗ Error reading from database: {e}")
        return None



# List existing Tables

In [4]:
# List existing tables
existing_tables = list_tables()

Found 1 table(s) in the database:

1. clientlist


# Import Client Telephone Numbers

In [5]:
client_list = pd.read_csv('407N 13th November 2025 csv.csv')
client_list.fillna(0, inplace=True)
client_list.columns = ['customer_Name', 'account_telephone_number', 'sales_telephone_number', 'mobile_number']
client_list

,customer_Name,account_telephone_number,sales_telephone_number,mobile_number
0,123456789,0,0,0
1,555,0,0,0862553507
2,A001,01 6793722,089 4241049,0
3,A002,01 6793722,086 3589622,0852243519
4,AA000,01 4964366,0,0862633794
...,...,...,...,...
2571,0,0,0,0
2572,0,0,0,0
2573,0,0,0,0
2574,0,0,0,0


# Insert Data Into Database

In [6]:
# Insert client_list dataframe into database
# Make sure client_list is defined before running this cell
if 'client_list' in locals() or 'client_list' in globals():
    create_and_insert_clientlist(client_list)
else:
    print("Error: 'client_list' dataframe not found. Please define it first.")

DataFrame Info:
Shape: (2576, 4)

Columns: ['customer_Name', 'account_telephone_number', 'sales_telephone_number', 'mobile_number']

Data types:
customer_Name               object
account_telephone_number    object
sales_telephone_number      object
mobile_number               object
dtype: object

First few rows:
  customer_Name account_telephone_number sales_telephone_number mobile_number
0     123456789                        0                      0             0
1           555                        0                      0    0862553507
2          A001               01 6793722            089 4241049             0
3          A002               01 6793722            086 3589622    0852243519
4         AA000               01 4964366                      0    0862633794

Dropping existing 'clientlist' table if it exists...
Creating 'clientlist' table...

SQL:

            CREATE TABLE clientlist (
                id SERIAL PRIMARY KEY,
        
                "customer_Name" TEXT,


# Verify Tables Created

In [7]:
# List tables to verify deletion
print("\n" + "="*50)
print("Remaining tables:")
print("="*50 + "\n")
list_tables()


Remaining tables:

Found 1 table(s) in the database:

1. clientlist


['clientlist']

# Import Call Log

In [ ]:
# Find and load all call log CSV files
files = glob.glob(CALL_LOG_PATTERN)
print(f"Found {len(files)} call log file(s)")

# Read and concatenate all CSV files
call_reports = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Remove the totals row if present
if call_reports['Call Time'].iloc[-1] == 'Totals':
    call_reports = call_reports.iloc[:-1]

# Rename column for consistency
call_reports.rename(columns={"From": "Caller ID"}, inplace=True)

# Add week labels and process caller information
call_reports = add_week_label(call_reports)

print(f"Loaded {len(call_reports)} call records")
print(f"Date range: {call_reports['Call Time'].min()} to {call_reports['Call Time'].max()}")
print(f"\nCustomer type distribution:")
print(call_reports['Type'].value_counts())

call_reports.head()

# Import Data from Database

In [9]:
# Import data from database
imported_data = import_clientlist_from_db()

Reading data from 'clientlist' table...
✓ Successfully loaded 2576 rows from database

DataFrame Info:
Shape: (2576, 5)
Columns: ['id', 'customer_Name', 'account_telephone_number', 'sales_telephone_number', 'mobile_number']

First few rows:
   id customer_Name account_telephone_number sales_telephone_number  \
0   1     123456789                        0                      0   
1   2           555                        0                      0   
2   3          A001               01 6793722            089 4241049   
3   4          A002               01 6793722            086 3589622   
4   5         AA000               01 4964366                      0   
5   6         AA001                  6179999             0879800153   
6   7         AAA00               0864055959              014605927   
7   8        AAA001                        0            087 6060551   
8   9         AB000               01 8646509             01 8646509   
9  10         AB001                        0     

C:\Users\irish\AppData\Local\Temp\ipykernel_9848\431660086.py:203: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [10]:
imported_data

,id,customer_Name,account_telephone_number,sales_telephone_number,mobile_number
0,1,123456789,0,0,0
1,2,555,0,0,0862553507
2,3,A001,01 6793722,089 4241049,0
3,4,A002,01 6793722,086 3589622,0852243519
4,5,AA000,01 4964366,0,0862633794
...,...,...,...,...,...
2571,2572,0,0,0,0
2572,2573,0,0,0,0
2573,2574,0,0,0,0
2574,2575,0,0,0,0


# Delete Tables if Necessary

In [33]:
# Delete the tables
delete_tables()
# Run the connection test
test_connection()

Dropping 'orders' table...
Dropping 'products' table...
Dropping 'customers' table...

✓ All tables deleted successfully!
Attempting to connect to PostgreSQL database...

✓ Connection successful!

PostgreSQL Database Version:
PostgreSQL 17.6 (Ubuntu 17.6-2.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0, 64-bit

Connected to database: tequila_ai_reporting

✓ Connection closed successfully


True